In [1]:
from pyspark.sql.functions import *

from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [2]:
df = spark.read.csv("original.csv", header=True,inferSchema=True)

In [3]:
df=df.where(df.id < 10)
#df.write.partitionBy("City").parquet(path="abcd")

In [5]:
df.write.partitionBy("City").parquet(path="folder/abc",mode="overwrite")

In [6]:
df_parq = spark.read.parquet("folder/abc")
import pyspark.sql.functions as sqlfun
from pyspark.sql  import types

In [7]:
df_parq.show(2)

+---+----------+---------+------+--------------------+---------+----------+----------+-------------+
| id|first_name|last_name|gender|            JobTitle|   Salary|  Latitude| Longitude|         City|
+---+----------+---------+------+--------------------+---------+----------+----------+-------------+
|  4|   Shannon|O'Griffin|  Male|Budget/Accounting...|$61489.23|44.5047212|38.1300171|Divnomorskoye|
|  8|   Goddart|    Flear|  Male|Desktop Support T...|$46116.36|45.1905186| 0.7423124|    Trélissac|
+---+----------+---------+------+--------------------+---------+----------+----------+-------------+
only showing top 2 rows



In [32]:
df_cleaned=df.withColumn("Salary",sqlfun.substring("Salary",2,10).cast(types.FloatType()))\
    .withColumn("gender",sqlfun.when(df.gender == "Female","F").otherwise("M"))
#df_gender= df.groupBy("gender").count()

In [33]:
#df_gender.show()
df_cleaned.show(2)

+---+----------+----------+------+---------+-------------------+--------+----------+-----------+
| id|first_name| last_name|gender|     City|           JobTitle|  Salary|  Latitude|  Longitude|
+---+----------+----------+------+---------+-------------------+--------+----------+-----------+
|  1|   Melinde| Shilburne|     F|Nowa Ruda|Assistant Professor|57438.18|50.5774075| 16.4967184|
|  2|  Kimberly|Von Welden|     F|   Bulgan|      Programmer II| 62846.6|48.8231572|103.5218199|
+---+----------+----------+------+---------+-------------------+--------+----------+-----------+
only showing top 2 rows



In [42]:
df_city = df_cleaned.groupBy(df_cleaned.City).agg(sum("Salary"),sqlfun.count("Salary"))

In [39]:
df_cleaned.where(df_cleaned.City == "Sułkowice").show()

+---+----------+---------+------+---------+--------------------+--------+----------+----------+
| id|first_name|last_name|gender|     City|            JobTitle|  Salary|  Latitude| Longitude|
+---+----------+---------+------+---------+--------------------+--------+----------+----------+
| 13|      Kerr|   Braden|     M|Sułkowice|Compensation Analyst|33432.99|49.8151822|19.3771749|
+---+----------+---------+------+---------+--------------------+--------+----------+----------+



In [43]:
df_city.show(2)

+---------+--------------+-------------+
|     City|   sum(Salary)|count(Salary)|
+---------+--------------+-------------+
|Sułkowice|33432.98828125|            1|
|  Klippan|   77039.46875|            1|
+---------+--------------+-------------+
only showing top 2 rows



In [28]:
df.show()
df.dtypes

+---+----------+----------+------+---------------+--------------------+-------+----------+-----------+
| id|first_name| last_name|gender|           City|            JobTitle| Salary|  Latitude|  Longitude|
+---+----------+----------+------+---------------+--------------------+-------+----------+-----------+
|  1|   Melinde| Shilburne|Female|      Nowa Ruda| Assistant Professor|7438.18|50.5774075| 16.4967184|
|  2|  Kimberly|Von Welden|Female|         Bulgan|       Programmer II| 2846.6|48.8231572|103.5218199|
|  3|    Alvera|  Di Boldi|Female|           NULL|                NULL|7576.52|39.9947462|116.3397725|
|  4|   Shannon| O'Griffin|  Male|  Divnomorskoye|Budget/Accounting...|1489.23|44.5047212| 38.1300171|
|  5|  Sherwood|   Macieja|  Male|      Mytishchi|            VP Sales|3863.09|      NULL| 37.6489954|
|  6|     Maris|      Folk|Female|Kinsealy-Drinan|      Civil Engineer| 101.16|53.4266145| -6.1644997|
|  7|     Masha|    Divers|Female|         Dachun|                NULL|50

[('id', 'int'),
 ('first_name', 'string'),
 ('last_name', 'string'),
 ('gender', 'string'),
 ('City', 'string'),
 ('JobTitle', 'string'),
 ('Salary', 'float'),
 ('Latitude', 'double'),
 ('Longitude', 'double')]